# Pytorch Implementation of EEGLearn - P. Bashivan

This notebook describes a short summary of Pytorch implementation of the models described in "Learning Representations from EEG with Deep Recurrent-Convolutional Neural Networks." Bashivan et al. at International conference on learning representations (2016).

The rest of the code is in the different python scripts of this repo.

All the codes have been inspired from the [original github](https://github.com/pbashivan/EEGLearn).

## Librairies Import

In [1]:
import numpy as np 
import scipy.io as sio
import torch
import os 

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

We adopted a recurrent-convolutional neural network to deal with the inherent structure of EEG data.
ConvNets were used to deal with variations in space and frequency domains due to their ability to
learn good two-dimensional representation of the data. Wherever needed, the extracted representations were fed into another layer to account for temporal variations in the data. We evaluated various
types of layers used for extracting temporal patterns, including convolutional and recurrent layers.
Essentially, we evaluated the following two primary approaches to the cognitive state classification problem.

1) Single-frame approach: a single image was constructed from spectral measurements
over the complete trial duration. The constructed image was then used as input to the ConvNet.

2)
Multi-frame approach: We divided each trial into 0.5 second windows and constructed an image over each time window, delivering 7 frames per trial (see section 4). The sequence of images was then used as input data to the recurrent-convolutional network. We used Lasagne1 to implement different architectures discussed in this paper. The code necessary for generating EEG images and building and training the networks discussed in this paper is available online2
.

## Loading the original Images 
The images have directly been taken from original implementation, given that they remain the same nevermind the implementation (Pytorch, Tensorflow, Theano).

In [2]:
Mean_Images = sio.loadmat("Sample Data/images.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Mean_Images)) 
Images = sio.loadmat("Sample Data/images_time.mat")["img"] #corresponding to the images mean for all the seven windows
print(np.shape(Images)) 
Label = (sio.loadmat("Sample Data/FeatureMat_timeWin")["features"][:,-1]-1).astype(int) #corresponding to the signal label (i.e. load levels).
print(np.shape(Label)) 
Patient_id = sio.loadmat("Sample Data/trials_subNums.mat")['subjectNum'][0] #corresponding to the patient id
print(np.shape(Patient_id))

(2670, 3, 32, 32)
(7, 2670, 3, 32, 32)
(2670,)
(2670,)


## Loading patient dataset 
From the total data, we select the images corresponding patient. 

In [3]:
print("Choose among the patient : "+str(np.unique(Patient_id)))

Choose among the patient : [ 1  2  3  4  6  7  8  9 10 11 12 14 15]


In [5]:
choosen_patient = 9

## Introduction: BasicCNN
First Implementation of a CNN on the Mean Images from each patient

In [6]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [7]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Mean_Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [8]:
res = TrainTest_Model(BasicCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=-1, opti='Adam', verbose=True)

Finished Training 
 loss: 0.000	Accuracy : 1.000		val-loss: 0.236	val-Accuracy : 0.975


## Maxpool CNN
Build the Max-pooling model performing a maxpool over the 7 parallel convnets.

In [9]:
train_part = 0.8
test_part = 0.2

batch_size = 32

In [29]:
EEG = EEGImagesDataset_time(label=Label[Patient_id==choosen_patient], image=Images[:,Patient_id==choosen_patient,:,:,:])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [30]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 0.705	Accuracy : 0.630		val-loss: 1.173	val-Accuracy : 0.625
[10,  45]	loss: 0.132	Accuracy : 0.932		val-loss: 0.732	val-Accuracy : 0.900
[15,  45]	loss: 0.239	Accuracy : 0.914		val-loss: 0.352	val-Accuracy : 0.850
[20,  45]	loss: 0.034	Accuracy : 0.994		val-loss: 0.253	val-Accuracy : 0.950
[25,  45]	loss: 0.001	Accuracy : 1.000		val-loss: 1.074	val-Accuracy : 0.850
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.151	val-Accuracy : 0.925
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.159	val-Accuracy : 0.925
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.124	val-Accuracy : 0.925
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 1.100	val-Accuracy : 0.925


## Temp CNN
FBuild the Conv1D model performing a convolution1D over the 7 parallel convnets.

In [17]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=45, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 0.435	Accuracy : 0.821		val-loss: 1.963	val-Accuracy : 0.725
[10,  45]	loss: 0.240	Accuracy : 0.895		val-loss: 2.726	val-Accuracy : 0.825
[15,  45]	loss: 0.001	Accuracy : 1.000		val-loss: 3.100	val-Accuracy : 0.875
[20,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.055	val-Accuracy : 0.875
[25,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.248	val-Accuracy : 0.875
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.325	val-Accuracy : 0.875
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.382	val-Accuracy : 0.875
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.427	val-Accuracy : 0.875
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.467	val-Accuracy : 0.875


## LSTM CNN
Build the LSTM model applying a RNN over the 7 parallel convnets outputs

In [21]:
EEG = EEGImagesDataset(label=Label[Patient_id==choosen_patient], image=Images[Patient_id==choosen_patient])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 7 but corresponding boolean dimension is 2670

In [23]:
EEG = EEGImagesDataset_time(label=Label[Patient_id==choosen_patient], image=Images[:,Patient_id==choosen_patient,:,:,:])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [24]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=45, learning_rate=0.0001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 1.345	Accuracy : 0.309		val-loss: 1.376	val-Accuracy : 0.200
[10,  45]	loss: 1.294	Accuracy : 0.302		val-loss: 1.361	val-Accuracy : 0.275
[15,  45]	loss: 1.202	Accuracy : 0.296		val-loss: 1.367	val-Accuracy : 0.300
[20,  45]	loss: 0.917	Accuracy : 0.593		val-loss: 1.055	val-Accuracy : 0.525
[25,  45]	loss: 0.672	Accuracy : 0.685		val-loss: 0.913	val-Accuracy : 0.575
[30,  45]	loss: 0.526	Accuracy : 0.784		val-loss: 0.809	val-Accuracy : 0.600
[35,  45]	loss: 0.388	Accuracy : 0.864		val-loss: 0.746	val-Accuracy : 0.650
[40,  45]	loss: 0.286	Accuracy : 0.926		val-loss: 0.959	val-Accuracy : 0.625
[45,  45]	loss: 0.195	Accuracy : 0.957		val-loss: 0.746	val-Accuracy : 0.650


## Mix CNN
Build the LSTM model applying a RNN and a CNN over the 7 parallel convnets outputs

In [25]:
EEG = EEGImagesDataset_time(label=Label[Patient_id==choosen_patient], image=Images[:,Patient_id==choosen_patient,:,:,:])

lengths = [int(len(EEG)*train_part+1), int(len(EEG)*test_part)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [26]:
print('Begin Training for Patient '+str(choosen_patient))
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=60, learning_rate=0.00001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  60]	loss: 1.368	Accuracy : 0.302		val-loss: 1.385	val-Accuracy : 0.300
[10,  60]	loss: 1.352	Accuracy : 0.296		val-loss: 1.387	val-Accuracy : 0.250
[15,  60]	loss: 1.340	Accuracy : 0.302		val-loss: 1.389	val-Accuracy : 0.275
[20,  60]	loss: 1.311	Accuracy : 0.531		val-loss: 1.369	val-Accuracy : 0.500
[25,  60]	loss: 1.196	Accuracy : 0.377		val-loss: 1.276	val-Accuracy : 0.375
[30,  60]	loss: 1.046	Accuracy : 0.525		val-loss: 1.147	val-Accuracy : 0.525
[35,  60]	loss: 0.903	Accuracy : 0.556		val-loss: 1.025	val-Accuracy : 0.500
[40,  60]	loss: 0.786	Accuracy : 0.605		val-loss: 0.932	val-Accuracy : 0.525
[45,  60]	loss: 0.672	Accuracy : 0.673		val-loss: 0.838	val-Accuracy : 0.600
[50,  60]	loss: 0.520	Accuracy : 0.778		val-loss: 0.706	val-Accuracy : 0.725
[55,  60]	loss: 0.347	Accuracy : 0.901		val-loss: 0.556	val-Accuracy : 0.775
[60,  60]	loss: 0.221	Accuracy : 0.932		val-loss: 0.430	val-Accuracy : 0.850
